In [1]:
import pandas as pd
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, precision_recall_curve, average_precision_score, mean_squared_error, r2_score, mean_absolute_error
from scipy.stats import pearsonr

import numpy as np

# preds

# Load the training set of meta-model
lipo_chemberta2_valid2 = pd.read_csv('./chemberta2/results/lipo/chemberta2_valid2_lipo_3_predictions.csv')
lipo_molformer_valid2 = pd.read_csv('./molformer/results/lipo/molformer_valid2_lipo_3.csv')
lipo_molbert_valid2 = pd.read_csv('./molbert/results/lipo/molbert_valid2_lipo_3.csv')

# Load the test data for each model
lipo_chemberta2_test = pd.read_csv('./chemberta2/results/lipo/chemberta2_test_lipo_3_predictions.csv')
lipo_molformer_test = pd.read_csv('./molformer/results/lipo/molformer_test_lipo_3.csv')
lipo_molbert_test = pd.read_csv('./molbert/results/lipo/molbert_test_lipo_3.csv')

train_mean = 2.1720992063492064
train_sd = 1.201255528709618

# features

lipo_chemberta2_features_valid2 = pd.read_csv('./chemberta2/features/lipo/chemberta2_valid2_lipo_3_features.csv')
lipo_chemberta2_features_test = pd.read_csv('./chemberta2/features/lipo/chemberta2_test_lipo_3_features.csv')

lipo_molformer_features_valid2 = pd.read_csv('./molformer/features/lipo/molformer_valid2_lipo_3_features.csv')
lipo_molformer_features_test = pd.read_csv('./molformer/features/lipo/molformer_test_lipo_3_features.csv')

lipo_molbert_features_valid2 = pd.read_csv('./molbert/features/lipo/molbert_valid2_lipo_3_features.csv')
lipo_molbert_features_test = pd.read_csv('./molbert/features/lipo/molbert_test_lipo_3_features.csv')

For Lipo (Regression)

In [2]:
# Preparing the actual and predicted values
# Chemberta2
lipo_chemberta_actual = lipo_chemberta2_test['target'] 
lipo_chemberta_pred = lipo_chemberta2_test['pred_raw']

# Molformer
lipo_molformer_actual = lipo_molformer_test['target']
lipo_molformer_pred = lipo_molformer_test['pred_raw']

# molbert
lipo_molbert_actual = lipo_molbert_test['target_raw']
lipo_molbert_pred = lipo_molbert_test['pred_raw']

In [3]:
# Calculating metrics
lipo_metrics_results = {}

for model_name, actual, pred in [("Chemberta2", lipo_chemberta_actual, lipo_chemberta_pred),
                                 ("Molformer", lipo_molformer_actual, lipo_molformer_pred),
                                 ("Molbert", lipo_molbert_actual, lipo_molbert_pred)]:
    lipo_metrics_results[model_name] = {
        "MAE": mean_absolute_error(actual, pred),
        "RMSE": np.sqrt(mean_squared_error(actual, pred)),
        "R2 Score": r2_score(actual, pred),
        "Correlation": pearsonr(actual, pred)[0]  # Only record the correlation coefficient
    }

lipo_metrics_results

{'Chemberta2': {'MAE': 0.46657646298682925,
  'RMSE': 0.6194209838770622,
  'R2 Score': 0.7336677436091222,
  'Correlation': 0.8681521305961759},
 'Molformer': {'MAE': 0.4598245026857143,
  'RMSE': 0.6138971242226391,
  'R2 Score': 0.7383967476487712,
  'Correlation': 0.8677855416135948},
 'Molbert': {'MAE': 0.5155634538071429,
  'RMSE': 0.6599043655043522,
  'R2 Score': 0.697716846760879,
  'Correlation': 0.8363791911807847}}

In [4]:
# standardized valid2 labels
lipo_y_ensemble_valid2 = (lipo_chemberta2_valid2['target'] - train_mean)/train_sd

# Create the features for the ensemble from the prediction probabilities of being in class 1
lipo_X_ensemble_valid2 = pd.concat([
    lipo_chemberta2_valid2['pred_z'] - lipo_y_ensemble_valid2,
    lipo_molformer_valid2['pred_z'] - lipo_y_ensemble_valid2, 
    lipo_molbert_valid2['pred_z'] - lipo_y_ensemble_valid2,
    # add features from training set
    lipo_chemberta2_valid2['pred_z'],
    lipo_molformer_valid2['pred_z'],
    lipo_molbert_valid2['pred_z']
], axis=1)

lipo_chemberta2_pred = lipo_chemberta2_valid2['pred_z']
lipo_molformer_pred = lipo_molformer_valid2['pred_z']
lipo_molbert_pred = lipo_molbert_valid2['pred_z']

# change feature names of the ensemble so that they are unique
lipo_X_ensemble_valid2.columns = ['residuals_chemberta', 'residuals_molformer', 'residuals_molbert', 'chemberta', 'molformer', 'molbert']

In [5]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Compute residuals
chemberta_y_residual = lipo_chemberta2_valid2['pred_z'] - lipo_y_ensemble_valid2
molformer_y_residual = lipo_molformer_valid2['pred_z'] - lipo_y_ensemble_valid2
molbert_y_residual = lipo_molbert_valid2['pred_z'] - lipo_y_ensemble_valid2

# For input features, we use pred_z and the other features
chemberta_X = pd.concat([lipo_chemberta2_valid2['pred_z'], 
                         lipo_chemberta2_features_valid2.iloc[:, 2:]], axis=1)
molformer_X = pd.concat([lipo_molformer_valid2['pred_z'], 
                         lipo_molformer_features_valid2.iloc[:, 1:]], axis=1)
molbert_X = pd.concat([lipo_molbert_valid2['pred_z'], 
                       lipo_molbert_features_valid2.iloc[:, 1:]], axis=1)

# Standardize each dataset
scaler_chemberta = StandardScaler().fit(chemberta_X)
scaler_molformer = StandardScaler().fit(molformer_X)
scaler_molbert = StandardScaler().fit(molbert_X)

chemberta_X_scaled = scaler_chemberta.transform(chemberta_X)
molformer_X_scaled = scaler_molformer.transform(molformer_X)
molbert_X_scaled = scaler_molbert.transform(molbert_X)

In [6]:
import pandas as pd

# Export valid2 probabilities
lipo_chemberta2_pred.to_csv('./processed_data/lipo_chemberta_pred.csv', index=False)
lipo_molformer_pred.to_csv('./processed_data/lipo_molformer_pred.csv', index=False)
lipo_molbert_pred.to_csv('./processed_data/lipo_molbert_pred.csv', index=False)

# Convert numpy arrays to pandas DataFrames and export
pd.DataFrame(chemberta_X_scaled).to_csv('./processed_data/lipo_chemberta_X.csv', index=False)
pd.DataFrame(molformer_X_scaled).to_csv('./processed_data/lipo_molformer_X.csv', index=False)
pd.DataFrame(molbert_X_scaled).to_csv('./processed_data/lipo_molbert_X.csv', index=False)

# Export BCE targets
chemberta_y_residual.to_csv('./processed_data/lipo_chemberta_y.csv', index=False)
molformer_y_residual.to_csv('./processed_data/lipo_molformer_y.csv', index=False)
molbert_y_residual.to_csv('./processed_data/lipo_molbert_y.csv', index=False)

# Export ensemble predictions
lipo_y_ensemble_valid2.to_csv('./processed_data/lipo_y_ensemble_valid2.csv', index=False)

In [7]:
# ChemBERTa
chemberta_X_test = pd.concat([lipo_chemberta2_test['pred_z'], 
                              lipo_chemberta2_features_test.iloc[:, 2:]], axis=1)

# MolFormer
molformer_X_test = pd.concat([lipo_molformer_test['pred_z'], 
                              lipo_molformer_features_test.iloc[:, 1:]], axis=1)

# MolBERT
molbert_X_test = pd.concat([lipo_molbert_test['pred_z'], 
                            lipo_molbert_features_test.iloc[:, 1:]], axis=1)

lipo_chemberta2_pred_test = lipo_chemberta2_test['pred_z']
lipo_molformer_pred_test = lipo_molformer_test['pred_z']
lipo_molbert_pred_test = lipo_molbert_test['pred_z']

# Scale the test data using the corresponding scalers
chemberta_X_test_scaled = scaler_chemberta.transform(chemberta_X_test)
molformer_X_test_scaled = scaler_molformer.transform(molformer_X_test)
molbert_X_test_scaled = scaler_molbert.transform(molbert_X_test)

In [8]:
import pandas as pd

# Export test predabilities
lipo_chemberta2_pred_test.to_csv('./processed_data/lipo_chemberta_pred_test.csv', index=False)
lipo_molformer_pred_test.to_csv('./processed_data/lipo_molformer_pred_test.csv', index=False)
lipo_molbert_pred_test.to_csv('./processed_data/lipo_molbert_pred_test.csv', index=False)

# Convert numpy arrays to pandas DataFrames and export test scaled data
pd.DataFrame(chemberta_X_test_scaled).to_csv('./processed_data/lipo_chemberta_X_test.csv', index=False)
pd.DataFrame(molformer_X_test_scaled).to_csv('./processed_data/lipo_molformer_X_test.csv', index=False)
pd.DataFrame(molbert_X_test_scaled).to_csv('./processed_data/lipo_molbert_X_test.csv', index=False)

# Export ensemble test class labels
lipo_y_ensemble_test = lipo_chemberta2_test['target']
lipo_y_ensemble_test.to_csv('./processed_data/lipo_y_ensemble_test.csv', index=False)